# How-to

This notebook serves as a practical guide to common questions users might have.

**Table of content**

- [Changing the parameters for an interaction](#Changing-the-parameters-for-an-interaction)
- [Writing your own interaction](#Writing-your-own-interaction)
- [Working with docking poses instead of MD simulations](#Working-with-docking-poses-instead-of-MD-simulations)
- [Using PDBQT files](#Using-PDBQT-files)



In [ ]:
import MDAnalysis as mda
import prolif as plf
import pandas as pd
import numpy as np

In [ ]:
u = mda.Universe(plf.datafiles.TOP, plf.datafiles.TRAJ)
lig = u.select_atoms("resname LIG")
prot = u.select_atoms("protein")
lmol = plf.Molecule.from_mda(lig)
pmol = plf.Molecule.from_mda(prot)

## Changing the parameters for an interaction

You can list all the available interactions as follow:

In [ ]:
plf.Fingerprint.list_available(show_hidden=True)

In this example, we'll redefine the hydrophobic interaction with a shorter distance.

You have the choice between overwriting the original hydrophobic interaction with the new one, or giving it an original name.

Let's start with a test case: with the default parameters, Y109 is interacting with our ligand.

In [ ]:
fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

### Overwriting the original interaction

You have to define a class that inherits one of the classes listed in the `prolif.interactions` module.

In [ ]:
class Hydrophobic(plf.interactions.Hydrophobic):
    def __init__(self):
        super().__init__(distance=4.0)

In [ ]:
fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

The interaction is not detected anymore. You can reset to the default interaction like so:

In [ ]:
class Hydrophobic(plf.interactions.Hydrophobic):
    pass

fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

### Reparameterizing an interaction with another name

The steps are identical to above, just give the class a different name:

In [ ]:
class CustomHydrophobic(plf.interactions.Hydrophobic):
    def __init__(self):
        super().__init__(distance=4.0)
        
fp = plf.Fingerprint()
fp.hydrophobic(lmol, pmol["TYR109.A"])

In [ ]:
fp.customhydrophobic(lmol, pmol["TYR109.A"])

In [ ]:
fp = plf.Fingerprint(["Hydrophobic", "CustomHydrophobic"])
fp.bitvector(lmol, pmol["TYR109.A"])

## Writing your own interaction

Before you dive into this section, make sure that there isn't already an interaction that could just be **reparameterized** to do what you want!

For this, the best is to check the section of the documentation corresponding to the `prolif.interactions` module. There are some generic interactions, like the `_Distance` class, if you just need to define two chemical moieties within a certain distance. Both the Hydrophobic, Ionic, and Metallic interactions inherit from this class!

With that being said, there are a few rules that you must respect when writing your own interaction:

* **Inherit the ProLIF Interaction class**
  
This class is located in `prolif.interactions.Interaction`. If for any reason you must inherit from another class, you can also define the `prolif.interactions._InteractionMeta` as a metaclass.

* **Naming convention**

Your class name must not start with `_` or be named `Interaction`. For non-symmetrical interactions, like hydrogen bonds or salt-bridges, the convention used here is to named the class after the function of the ligand. For example, the class `HBDonor` detects if a ligand acts as a hydrogen bond donor, and the class `Cationic` detects if a ligand acts as a cation.

* **Define a `detect` method**

This method takes exactly two positional arguments (and as many named arguments as you need): a ligand Residue or Molecule and a protein Residue or Molecule (in this order).

* **Return values for the `detect` method**

You must return the following items in this order: 

  * `True` or `False` for the detection of the interaction
  * The index of the ligand atom, or None if not detected
  * The index of the protein atom, or None if not detected

In [ ]:
from scipy.spatial import distance_matrix

class CloseContact(plf.interactions.Interaction):
    def detect(self, res1, res2, threshold=2.0):
        dist_matrix = distance_matrix(res1.xyz, res2.xyz)
        mask = dist_matrix <= threshold
        if mask.any():
            res1_i, res2_i = np.where(mask)
            # return the first solution
            return True, res1_i[0], res2_i[0]
        return False, None, None

fp = plf.Fingerprint()
fp.closecontact(lmol, pmol["ASP129.A"])

By default, the fingerprint will modify all interaction classes to only return the boolean value. To get the atom indices you must choose one of the following options:

* Call `fp.to_dataframe(return_atoms=True)`

* Use the `return_atoms=True` argument when calling the `generate` method:

In [ ]:
ifp = fp.generate(lmol, pmol, return_atoms=True)
# check the interactino between the ligand and ASP129
ifp[(plf.ResidueId("LIG", 1, "G"),
     plf.ResidueId("ASP", 129, "A"))]

* Use the `__wrapped__` argument when calling the class as a fingerprint method:

In [ ]:
fp.closecontact.__wrapped__(lmol, pmol["ASP129.A"])

* Use the `bitvector_atoms` method instead of `bitvector`:

In [ ]:
fp = plf.Fingerprint(["CloseContact"])
bv, lig_ix, prot_ix = fp.bitvector_atoms(lmol, pmol["ASP129.A"])
bv, lig_ix, prot_ix

* Directly use your class:

In [ ]:
cc = CloseContact()
cc.detect(lmol, pmol["ASP129.A"])

## Working with docking poses instead of MD simulations

ProLIF currently provides file readers for MOL2, SDF and PDBQT files. The API is slightly different compared to the quickstart example but the end result is the same.

Please note that this part of the tutorial is only suitable for interactions between one protein and several ligands, or in more general terms, between one molecule with multiple residues and one molecule with a single residue. This is not suitable for protein-protein or DNA-protein interactions.

Let's start by loading the protein. Here I'm using a PDB file but you can use any format supported by MDAnalysis as long as it contains explicit hydrogens.

In [ ]:
# load protein
prot = mda.Universe(plf.datafiles.datapath / "vina" / "rec.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

### Using an SDF file

In [ ]:
# load ligands
path = str(plf.datafiles.datapath / "vina" / "vina_output.sdf")
lig_suppl = list(plf.sdf_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe()
df

Please note that converting the `lig_suppl` to a list is optionnal (and maybe not suitable for large files) as it will load all the ligands in memory, but it's nicer to track the progression with the progress bar.

If you want to calculate the Tanimoto similarity between your docked poses and a reference ligand, here's how to do it.

We first need to generate the interaction fingerprint for the reference, and concatenate it to the previous one

In [ ]:
# load the reference
ref = mda.Universe(plf.datafiles.datapath / "vina" / "lig.pdb")
ref = plf.Molecule.from_mda(ref)
# generate IFP
fp.run_from_iterable([ref], prot)
df0 = fp.to_dataframe()
df0.rename({0: "ref"}, inplace=True)
# drop the ligand level on both dataframes
df0.columns = df0.columns.droplevel(0)
df.columns = df.columns.droplevel(0)
# concatenate them
df = (pd.concat([df0, df])
        .fillna(False)
        .sort_index(axis=1, level=0,
                    key=lambda index: [plf.ResidueId.from_string(x) for x in index]))
df

Lastly, we can convert the dataframe to a list of RDKit bitvectors to finally compute the Tanimoto similarity between our reference pose and the docking poses generated by Vina:

In [ ]:
from rdkit import DataStructs

bvs = plf.to_bitvectors(df)
for i, bv in enumerate(bvs[1:]):
    tc = DataStructs.TanimotoSimilarity(bvs[0], bv)
    print(f"{i}: {tc:.3f}")

Interestingly, the best scored docking pose (#0) isn't the most similar to the reference (#5)

### Using a MOL2 file

The input mol2 file can contain multiple ligands in different conformations.

In [ ]:
# load ligands
path = plf.datafiles.datapath / "vina" / "vina_output.mol2"
lig_suppl = list(plf.mol2_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe()
df

### Using PDBQT files

The typical use case here is getting the IFP from AutoDock Vina's output. It requires a few additional steps and informations compared to other formats like MOL2, since the PDBQT format gets rid of most hydrogen atoms and doesn't contain bond order information.

The prerequisites for a successfull usage of ProLIF in this case is having external files that contain bond orders and formal charges for your ligand (like SMILES, SDF or MOL2), or at least a file with explicit hydrogen atoms. 

Please note that your PDBQT input must have a single model per file (this is required by MDAnalysis). Splitting a multi-model file can be done using the `vina_split` command-line tool that comes with AutoDock Vina: `vina_split --input vina_output.pdbqt`

Let's start by loading our "template" file with bond orders. It can be a SMILES string, MOL2, SDF file or anything supported by RDKit.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

template = Chem.MolFromSmiles("C[NH+]1CC(C(=O)NC2(C)OC3(O)C4CCCN4C(=O)C"
                              "(Cc4ccccc4)N3C2=O)C=C2c3cccc4[nH]cc(c34)CC21")
template

Next, we'll use the PDBQT supplier which loads each file from a list of paths, and assigns bond orders and charges using the template. The template and PDBQT file must have the exact same atoms, **even hydrogens**, otherwise no match will be found. Since PDBQT files partially keep the hydrogen atoms, we have the choice between:

* Manually selecting where to add the hydrogens on the template, do the matching, then add the remaining hydrogens (not covered here)
* Or just remove the hydrogens from the PDBQT file, do the matching, then add all hydrogens.

This last option will delete the coordinates of your hydrogens atoms and replace them by the ones generated by RDKit, but unless you're working with an exotic system this should be fine.

For the protein, there's usually no need to load the PDBQT that was used by Vina. The original file that was used to generate the PDBQT can be used directly, but **it must contain explicit hydrogen atoms**:

In [ ]:
# load ligands
pdbqt_files = sorted(plf.datafiles.datapath.glob("vina/*.pdbqt"))
lig_suppl = list(plf.pdbqt_supplier(pdbqt_files, template))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe()
df